# Information Retrieval
 
La Recuperación de Información (Information Retrieval, IR) es la ciencia que permite encontrar información en datos de naturaleza no estructurada, dada una necesidad de información.

Imaginad que queremos tener un buscador de libros.

<img src=https://i.imgur.com/CLRu4C7.png>

> Se plantean algunos retos...

El diagrama de una arquitectura genérica para un sistema de IR que veíamos:

<img src=https://www.tutorialspoint.com/natural_language_processing/images/relevant_output_about_information.jpg>

## Partes de la arquitectura

- The User: ....
- Query: "comics de batman"
- Query operations: Transforma la query de texto, en alguna representación común a la de los documentos.
- Document collection: ...
- Indexer: Representación común de los documentos.
- Retrieval System + Rankeo: Modelo o sistema, que calcula que documentos son más relevantes, dada la query del usuario.
- Feedback: explícito o implícito 

## ¿Como lo hacemos?

Vamos a ver algunos ejemplos de implementaciones sencillas de sistemas de IR.

## Recursos

* [Information Retrieval Book](https://nlp.stanford.edu/IR-book/) Recomendadíssimo.
* [Papers](https://ai.google/research/pubs?area=InformationRetrievalandtheWeb), muchos papers
* [Neural IR Recap](https://arxiv.org/pdf/1705.01509.pdf)

# Boolean Retrieval

Modelo más simple de Information Retrieval (IR). Basado en lógica booleana y teoría de conjuntos, los documentos estarán representados por un vector de tamaño el vocabulario del corpus, y tendrá como valores 1 o 0 (Term Presence).

En esencia, es un bag-of-words donde solo se considera la presencia (o ausencia) de las palabras que conforman el vocabulario.

Las queries estarán también expresadas mediante lógica booleana (otro vector similar al de un documento).

Nuestro corpus - o conjunto de documentos - estará representado como una matriz de documento-término

In [ ]:
import numpy as np

In [ ]:
V = 5  # número de palabras en el vocabulario
D = 3  # número de documentos en el corpus

a = np.random.randint(2, size=(D,V), dtype="bool")
a.shape

In [ ]:
# Matriz documento-término con el Term Presence como feature weight
a

In [ ]:
a[0,:]

In [ ]:
a[:,1]

Con numpy tambien podemos hacer uso de puertas lógicas

https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.logic.html

In [ ]:
np.logical_and(a[0,:], a[1,:])

In [ ]:
np.logical_and(np.logical_and(a[0,:], a[1,:]), np.logical_not(a[2,:]))

Por ejemplo, si quiero todos los documentos que contengan el termino 0 o 1

In [ ]:
feat_vect = np.array([False, False, True, False, False])  # query de prueba

In [ ]:
a

In [ ]:
np.dot(a, feat_vect)

### Sparse Representations

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
a_sparse = csr_matrix(a)
a_sparse[:].nonzero()

In [ ]:
from sys import getsizeof
print(getsizeof(a), getsizeof(a_sparse))

## Inverted Indexes

In [ ]:
documents = [
    "Julio Cesar era un emperador romano",
    "La ensalada Cesar lleva tomate lechuga y pollo",
    "El restaurante Casa Cesar ofrece una variedad de platos muy grande",
    "Las ensaladas son muy sanas",
    "A Cesar le gustan mucho las ensaladas",
    "Cesar era el emperador más querido. A Cesar le construyeron un museo. Los hombres y mujeres aclamaban a Cesar"
]

In [ ]:
vocabulary = set([t for doc in documents for t in doc.split(" ")])
w2id = {k:i for i, k in enumerate(vocabulary)}

In [ ]:
doc_matrix = np.zeros((len(documents), len(vocabulary)))
doc_matrix.shape

In [ ]:
for i, doc in enumerate(documents):
    for t in doc.split(" "):
        doc_matrix[i,w2id[t]]+=1
doc_matrix

In [ ]:
inverted_index = {}
for id_doc, doc in enumerate(documents):
    for t in doc.split(" "):
        if t in inverted_index:
            inverted_index[t].append(id_doc)
        else:
            inverted_index[t] = [id_doc]
inverted_index

In [ ]:
# Una pequeña mejora. Se indica no solo el documento si no también "dónde" se encuentra en dichos documentos
inverted_index = {}
for id_doc, doc in enumerate(documents):
    for pos, t in enumerate(doc.split(" ")):
        if t in inverted_index:
            inverted_index[t].append((id_doc,pos))
        else:
            inverted_index[t] = [(id_doc,pos)]
inverted_index

In [ ]:
query = ['ensalada', 'Cesar']

In [ ]:
# Si ALGUNO (OR) de los términos aparece en el documento -> print(doc)
for q_item in query:
    if q_item in inverted_index:
        for doc_id in inverted_index[q_item]:
            print(documents[doc_id[0]])

In [ ]:
# Si AMBOS (AND) de los términos aparece en el documento -> print(doc)
possible_docs ={}
for q_item in query:
    if q_item in inverted_index:
        for doc_id in inverted_index[q_item]:
            if doc_id[0] in possible_docs:
                possible_docs[doc_id[0]]+=1
            else:
                possible_docs[doc_id[0]]=1
true_docs = [doc_id for doc_id, count in possible_docs.items() if count == len(query)]
for doc in true_docs:
    print(documents[doc])

# Vector Spaces 

<img src=http://blog.christianperone.com/wp-content/uploads/2013/09/vector_space.png width=450px>

A diferencia del modelo booleano, las componentes en los vectores de cada documento tendrán un valor distinto al Term Presence. Dicho peso puede ser el que elijamos (por ejemplo, Term Frequency, o TF-IDF).

Pese a que el estado del arte muestra que modelos basados en Deep Learning arrojan mejores resultados, muy posiblemente la mayoría de sistemas de IR funcionen con alguna aproximación a este tipo de algoritmos.

Recordemos: Term Frequency y Document Frequency.

## TF y DF

<img src=https://i.imgur.com/9o3G6Ia.png>

## TF-IDF

Term Frequency - Inverse Document Frequency.

El weigth dado por el algoritmo tf-idf equivale a cuantas más veces aparezca una palabra en un documento, y cuantas menos veces aparezca en otros documentos, más importante será esa palabra para ese documento.

In [ ]:
nb_docs = len(documents)
vocab_size = len(vocabulary)

In [ ]:
vector_docs = np.zeros(shape=(nb_docs, vocab_size))
vector_docs.shape

## Term Frequency, algunas maneras de calcularlo

In [ ]:
# Term Presence (boolean TF)
tp_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    for t in doc.split(" "):
        tp_w[i, w2id[t]] = 1
print(tp_w)

In [ ]:
# TF
tf_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    for t in doc.split(" "):
        tf_w[i, w2id[t]] += 1
print(tf_w)

In [ ]:
# TF normalizado
tf_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    for t in doc.split(" "):
        tf_w[i, w2id[t]] += 1
    tf_w[i,:] /=len(doc.split(" "))
print(tf_w)

In [ ]:
# TF log
def apply_log(a, eps=0.1):
    a = a if a > 0 else eps
    return np.log(1+a)
vlog = np.vectorize(apply_log)

tf_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    for t in doc.split(" "):
        tf_w[i, w2id[t]] += 1
tf_w = vlog(tf_w)
print(tf_w)

In [ ]:
from collections import Counter

In [ ]:
# TF Augmented
def apply_augmented(a, max_w):
    return 0.5+((0.5*a)/max_w)
vaug = np.vectorize(apply_augmented)

tf_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    tokens = doc.split(" ")
    max_w = Counter(tokens).most_common(1)[0][1]
    for t in tokens:
        tf_w[i, w2id[t]] += 1
    tf_w[i,:] = vaug(tf_w[i,:], max_w)

print(tf_w)

## IDF, algunas formas de calcularlo

El IDF se computa como el numero de documentos dividido por el numero de veces que aparece el termino t en un la colección de documentos.

In [ ]:
from math import log

In [ ]:
# DF
df_dict = {w:0 for w in vocabulary}
for doc in documents:
    for w in df_dict.keys():
        if w in doc.split(" "):
            df_dict[w]+=1.0
print(df_dict)

In [ ]:
# Basic IDF
idf_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    tokens = doc.split(" ")
    for t in tokens:
        idf_w[i, w2id[t]] = log(nb_docs/df_dict[t])
print(idf_w)

In [ ]:
# Smooth IDF
idf_w = np.zeros(shape=(nb_docs, vocab_size))
for i, doc in enumerate(documents):
    tokens = doc.split(" ")
    for t in tokens:
        idf_w[i, w2id[t]] = log(1+(1+nb_docs/1+df_dict[t]))
print(idf_w)

## TF-IDF

In [ ]:
tf_idf_w = tf_w * idf_w
tf_idf_w.shape

In [ ]:
tf_idf_w[5,:]

## Feature Matrix

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(tf_idf_w, columns=vocabulary)

In [ ]:
documents

## Esto ya lo conocíamos (TfIdfVectorizer)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
docs = np.array(documents)
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(docs)


In [ ]:
pd.DataFrame(feature_matrix.toarray(), columns=tfidf.get_feature_names())

## Plotting documents

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tfidf_low_dim = TSNE(n_components=2).fit_transform(tf_idf_w)

In [ ]:
maxx = max(tfidf_low_dim[:, 0])+0.0001
minx = min(tfidf_low_dim[:, 0])-0.0001
maxy = max(tfidf_low_dim[:, 1])+0.0001
miny = min(tfidf_low_dim[:, 1])-0.0001

fig, ax = plt.subplots(figsize=(15, 8))

plt.scatter(tfidf_low_dim[:, 0], tfidf_low_dim[:, 1], cmap=plt.cm.Spectral)
plt.axis([minx, maxx, miny, maxy])
for i, txt in enumerate(np.arange(nb_docs).tolist()):
    ax.annotate(txt, (tfidf_low_dim[:, 0][i],tfidf_low_dim[:, 1][i]))
plt.show()

## Querying the model


![](http://blog.christianperone.com/wp-content/uploads/2013/09/cosinesimilarityfq1.png)



In [ ]:
query = 'ensalada Cesar sanas'

In [ ]:
q_w = np.zeros(shape=(1, vocab_size))

for t in query.split(" "):
    if t in w2id:
        q_w[0, w2id[t]] += 1
q_tf = q_w
q_tf.shape

In [ ]:
q_idf = np.zeros(shape=(1, vocab_size))

tokens = query.split(" ")
for t in tokens:
    if t in w2id:
        q_idf[0, w2id[t]] = log((nb_docs/df_dict[t]))
q_idf

In [ ]:
query_rep = q_tf * q_idf
query_rep

### Cosine similarity

Visualización de Cosine Similarity


![](https://lh4.googleusercontent.com/SodVc3Xo77b8LhEjqXymSaA-bI-kQdPeY8uG-J0wSSp5q-pxVAf_rPMUX9Y)




In [ ]:
def cos_similarity(x, y):
    numerator = np.sum(x*y, axis=-1)
    a = np.sqrt(np.sum(x**2, axis=-1))
    b = np.sqrt(np.sum(y**2, axis=-1))
    denominator = a*b
    return numerator/denominator

In [ ]:
for i in range(nb_docs):
    print(documents[i], cos_similarity(query_rep, tf_idf_w[i,:]))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(query_rep, tf_idf_w)